In [16]:
# --- 1. 기본 설정 및 라이브러리 임포트 ---

# 데이터 분석 및 처리를 위한 필수 라이브러리
import pandas as pd
import numpy as np
import random
import os, sys
from datetime import datetime

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from scipy import stats
from IPython.display import display

# 모델링 및 기계 학습을 위한 라이브러리
import lightgbm as lgb                              # LightGBM 모델
from sklearn.model_selection import StratifiedKFold, TimeSeriesSplit # 교차 검증을 위한 K-폴드 및 시계열 분할
from sklearn.preprocessing import LabelEncoder      # 범주형 데이터 인코딩
from sklearn.metrics import mean_squared_error      # 평가 지표 (RMSE 계산용)
from sklearn.cluster import KMeans                  # K-평균 군집화
from sklearn.preprocessing import StandardScaler    # 데이터 스케일링

# 하이퍼파라미터 최적화를 위한 라이브러리
import optuna

# 기타 유틸리티
import gc                                           # 가비지 컬렉션 (메모리 관리)
from tqdm import tqdm                               # 작업 진행률 표시
import warnings
warnings.filterwarnings('ignore')                   # 불필요한 경고 메시지 무시

# 한글 폰트 설정
import matplotlib.font_manager as fm
try:
    # 나눔고딕 폰트 경로 설정 (경로는 환경에 따라 조정 필요)
    font_path = '../../font/NanumFont/NanumGothic.ttf'
    
    # 폰트가 존재하는지 확인 후 설정
    if os.path.exists(font_path):
        fe = fm.FontEntry(fname=font_path, name='NanumGothic')
        fm.fontManager.ttflist.insert(0, fe)
        plt.rcParams.update({'font.size': 12, 'font.family': 'NanumGothic'})
    else:
        print("나눔고딕 폰트를 찾을 수 없어 기본 폰트로 설정됩니다.")
except Exception as e:
    print(f"폰트 설정 중 오류 발생: {e}")
    pass


# 로거(실행 기록 로그 저장) 임포트
src_path = os.path.abspath(os.path.join(os.getcwd(), "../../src/log"))
sys.path.insert(0, src_path)
from logger import Logger
print("Logger 모듈 로드 성공.")

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_DIR = '../../data/logs/price_prediction_6_logs'
LOG_FILENAME = f"price_prediction_{timestamp}.log"
LOG_PATH = os.path.join(LOG_DIR, LOG_FILENAME)
os.makedirs(LOG_DIR, exist_ok=True)
logger = Logger(log_path=LOG_PATH)


# 데이터 경로 설정
RAW_DIR             = '../../data/processed/clean_data'                 # 원본 데이터 디렉토리
TRAIN_FILENAME      = 'train.csv'                                       # 훈련 데이터 파일명
TEST_FILENAME       = 'test.csv'                                        # 테스트 데이터 파일명
TRAIN_PATH          = os.path.join(RAW_DIR, TRAIN_FILENAME)             # 훈련 데이터 경로
TEST_PATH           = os.path.join(RAW_DIR, TEST_FILENAME)              # 테스트 데이터 경로
SUBMISSION_DIR      = '../../data/processed/submissions'                # 제출 파일 디렉토리
SUBMISSION_FILENAME = 'price_prediction_6_submission.csv'               # 제출 파일명
SUBMISSION_PATH     = os.path.join(SUBMISSION_DIR, SUBMISSION_FILENAME) # 제출 파일 경로
MODEL_DIR           = '../../model'                                     # 모델 저장 디렉토리
MODEL_FILENAME      = 'price_prediction_6_model.pkl'                    # 모델 파일명
MODEL_PATH          = os.path.join(MODEL_DIR, MODEL_FILENAME)           # 모델 경로

# 결과 저장 디렉토리 생성
os.makedirs(SUBMISSION_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

logger.start_redirect()
logger.write("="*50)
logger.write(">> < 아파트 가격 예측 모델링 시작 >\n\n")
logger.write(">> [1단계 완료] 라이브러리, 경로, 로거 초기화 성공!")

In [17]:
# --- 2. 데이터 로드 및 초기 전처리 (수정 완료) ---
try:
    logger.write(">> [2단계 시작] 데이터 로드를 시작합니다...")
    
    # CSV 파일로부터 훈련 및 테스트 데이터를 로드합니다.
    train_df = pd.read_csv(TRAIN_PATH)
    test_df = pd.read_csv(TEST_PATH)

    # 훈련 데이터와 테스트 데이터를 하나로 합칩니다.
    df = pd.concat([train_df, test_df], axis=0).reset_index(drop=True)
    logger.write(">> 데이터 로드 및 병합 완료.")
    
    # TimeSeriesSplit을 위해 '계약일자' 컬럼만으로 시간순 정렬합니다.
    df = df.sort_values(by=['계약일자']).reset_index(drop=True)
    logger.write(">> TimeSeriesSplit을 위해 데이터를 시간순으로 정렬했습니다.")
    
except Exception as e:
    logger.write(f">> [오류] 2단계 데이터 로드 및 전처리 중 문제 발생: {e}", print_error=True)

In [ ]:
# --- 3. 피처 엔지니어링 (Feature Engineering) ---
# 기존 데이터를 바탕으로 모델 성능을 높일 새로운 특성을 생성합니다.

try:
    logger.write(">> [3단계 시작] 피처 엔지니어링을 시작합니다...")
    
    # '계약년월'을 '계약년'과 '계약월'로 분리합니다.
    df['계약년'] = df['계약년월'].astype(str).str[:4].astype(int)
    df['계약월'] = df['계약년월'].astype(str).str[4:].astype(int)
    
    # 건물의 나이(연식)을 계산합니다.
    df['건물나이'] = df['계약년'] - df['연식']
    
    # 거래 시점(연도와 월의 조합)을 나타내는 특성을 생성합니다.
    df['거래시점'] = (df['계약년'] - 2007) * 12 + df['계약월']
    
    # --- K-평균 군집화 (K-Means Clustering) ---
    try:
        logger.write(">> K-평균 군집화를 시작합니다...")
        cluster_cols = ['좌표X', '좌표Y', '건물나이', '전용면적', '층']
        
        scaler = StandardScaler()
        df_scaled = scaler.fit_transform(df[cluster_cols])
        
        kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)
        df['아파트군집'] = kmeans.fit_predict(df_scaled)
        
        logger.write(f">> K-평균 군집화 완료. '{len(cluster_cols)}'개 특성을 사용해 10개의 '아파트군집' 생성.")

    except Exception as e:
        logger.write(f">> [오류] K-평균 군집화 중 문제 발생: {e}", print_error=True)
    
    # --- 고급 피처 엔지니어링 ---
    logger.write(">> 고급 피처 엔지니어링을 시작합니다...")

    # 계약월의 주기성을 나타내는 sin/cos 변환 피처 생성
    df['계약월_sin'] = np.sin(2 * np.pi * df['계약월']/12)
    df['계약월_cos'] = np.cos(2 * np.pi * df['계약월']/12)
    logger.write(">> 1. 시간 관련(sin/cos) 피처 생성 완료.")
    
    # 교통 편의성 가중합 특성
    df['가중지하철'] = df['반경_1km_지하철역_수'] * 1.0 + df['반경_500m_지하철역_수'] * 1.5 + df['반경_300m_지하철역_수'] * 2.0
    df['가중버스'] = df['반경_1km_버스정류장_수'] * 1.0 + df['반경_500m_버스정류장_수'] * 1.5 + df['반경_300m_버스정류장_수'] * 2.0
    logger.write(">> 2. 교통 관련 가중합 특성 생성 완료.")

    # 면적당 가격 기반 통계 피처 생성
    train_df_temp = df[df['target'].notna()].copy()
    train_df_temp['면적당가격'] = np.log1p(train_df_temp['target']) / train_df_temp['전용면적']

    # --- [핵심 수정] Named Aggregation을 사용하여 중복 컬럼 문제 해결 ---
    # 법정동 및 자치구별 면적당 가격의 평균과 표준편차 계산
    dong_price_stats = train_df_temp.groupby('법정동').agg(
        동별_평균면적당가격=('면적당가격', 'mean'),
        동별_면적당가격편차=('면적당가격', 'std')
    ).reset_index()
    
    gu_price_stats = train_df_temp.groupby('자치구').agg(
        구별_평균면적당가격=('면적당가격', 'mean'),
        구별_면적당가격편차=('면적당가격', 'std')
    ).reset_index()
    
    # 전체 데이터프레임에 통계 피처 병합
    df = pd.merge(df, dong_price_stats, on='법정동', how='left')
    df = pd.merge(df, gu_price_stats, on='자치구', how='left')
    logger.write(">> 3. 면적당 가격 관련 통계 피처 생성 및 병합 완료.")

    # 상호작용 특성 생성
    df['면적_x_나이'] = df['전용면적'] * df['건물나이']
    df['면적_x_층'] = df['전용면적'] * df['층']
    df['강남_x_면적'] = df['강남3구여부'] * df['전용면적']
    logger.write(">> 4. 상호작용 특성 생성 완료.")

    # 생성된 통계 피처의 결측치 처리
    fill_na_cols = ['동별_평균면적당가격', '동별_면적당가격편차', '구별_평균면적당가격', '구별_면적당가격편차']
    
    # df에 실제로 존재하는 통계 컬럼만 선택하여 결측치를 채웁니다.
    existing_cols = [col for col in fill_na_cols if col in df.columns]
    
    if existing_cols:
        # 훈련 데이터(train_df_temp)의 평균으로만 결측치를 채워 데이터 유출을 방지합니다.
        mean_vals = train_df_temp[existing_cols].mean()
        df[existing_cols] = df[existing_cols].fillna(mean_vals)
        logger.write(">> 5. 통계 피처 결측치 처리 완료.")
    else:
        logger.write(">> 5. 생성된 통계 피처에 결측치가 없어 처리를 건너뜁니다.")

    logger.write(">> 고급 피처 엔지니어링 완료.")
    
    logger.write(">> [3단계 완료] 피처 엔지니어링 성공.")
    
except Exception as e:
    logger.write(f">> [오류] 3단계 피처 엔지니어링 중 문제 발생: {e}", print_error=True)

In [22]:
# --- 4. 데이터 스케일링 및 인코딩 ---

try:
    logger.write(">> [4단계 시작] 데이터 스케일링 및 인코딩을 시작합니다...")

    # --- 로그 변환 (Log Transformation) ---
    # 타겟 변수인 'target'(실거래가)의 분포를 정규분포에 가깝게 만들어 모델 성능을 안정시킵니다.
    # np.log1p는 np.log(x + 1)과 같으며, 0 또는 매우 작은 값에 대한 로그 계산 시 발생할 수 있는 오류를 방지합니다.
    df['target'] = np.log1p(df['target'])
    logger.write(">> 타겟 변수(target) 로그 변환 완료.")

    # --- 라벨 인코딩 (Label Encoding) ---
    # '아파트명', '도로명', '법정동'과 같은 문자열(범주형) 데이터를 모델이 이해할 수 있는 숫자형 데이터로 변환합니다.
    categorical_features = df.select_dtypes(include=['object']).columns
    for col in categorical_features:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
    logger.write(">> 범주형 데이터 라벨 인코딩 완료.")
    
except Exception as e:
    logger.write(f">> [오류] 4단계 스케일링 및 인코딩 중 문제 발생: {e}", print_error=True)

In [23]:
# --- 5. 최종 데이터 준비 ---
# 전처리와 피처 엔지니어링이 완료된 데이터를 다시 훈련 세트와 테스트 세트로 분리합니다.

try:
    logger.write(">> [5단계 시작] 최종 데이터 분리를 시작합니다...")
    
    # 'target' 컬럼의 결측치 여부를 기준으로 훈련 데이터와 테스트 데이터를 나눕니다.
    train_df = df[df['target'].notna()]
    test_df = df[df['target'].isna()]
    
    # 테스트 데이터에서는 더 이상 필요 없는 'target' 컬럼을 제거합니다.
    test_df = test_df.drop('target', axis=1)

    # 모델의 입력(X)과 타겟(y)을 정의합니다.
    # 'target'과 날짜 관련 컬럼은 입력에서 제외합니다.
    X_train = train_df.drop(['target', '계약년월'], axis=1)
    y_train = train_df['target']
    X_test = test_df.drop(['계약년월'], axis=1)
    
    logger.write(f">> 최종 데이터 분리 완료. 훈련 데이터: {X_train.shape}, 테스트 데이터: {X_test.shape}")
    logger.write(">> [5단계 완료] 최종 데이터 준비 성공.")
    
except Exception as e:
    logger.write(f">> [오류] 5단계 최종 데이터 준비 중 문제 발생: {e}", print_error=True)

In [24]:
# --- 6. 특성 선택 (Feature Selection) ---
# LightGBM 모델을 한 번 실행하여 모델 성능에 중요하게 기여하는 특성들을 선별합니다.

try:
    logger.write(">> [6단계 시작] 특성 선택을 시작합니다...")
    
    # 임시 LightGBM 모델을 훈련하여 특성 중요도를 계산합니다.
    temp_model = lgb.LGBMRegressor(random_state=42)
    temp_model.fit(X_train, y_train)
    
    # 특성 중요도가 0 이상인, 즉 모델이 한 번이라도 사용한 특성들만 선택합니다.
    feature_importances = pd.Series(temp_model.feature_importances_, index=X_train.columns)
    selected_features = feature_importances[feature_importances > 0].index.tolist()
    
    # 선택된 특성들만 사용하여 훈련 및 테스트 데이터를 다시 구성합니다.
    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]
    
    logger.write(f">> 특성 선택 완료. 총 {len(X_train.columns)}개 중 {len(selected_features)}개 선택됨.")
    logger.write(">> [6단계 완료] 특성 선택 성공.")
    
except Exception as e:
    logger.write(f">> [오류] 6단계 특성 선택 중 문제 발생: {e}", print_error=True)

In [ ]:
# --- 7. 하이퍼파라미터 최적화 (Optuna) ---
# 모델의 성능을 극대화하기 위해 최적의 하이퍼파라미터 조합을 탐색합니다.

try:
    # Optuna가 최적의 하이퍼파라미터를 찾기 위해 반복적으로 실행하는 함수(objective)를 정의합니다.
    def objective(trial, X, y):
        # 테스트할 하이퍼파라미터의 범위를 정의합니다.
        param = {
            'objective': 'regression_l1', 'metric': 'rmse',
            'n_estimators': 2000, 
            'verbosity': -1, 'boosting_type': 'gbdt', 'seed': 42,
            'device': 'cuda', 'n_jobs': -1,
            
            # Optuna의 trial 객체를 통해 탐색할 파라미터 값의 범위를 제안받습니다.
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05),
            'num_leaves': trial.suggest_int('num_leaves', 30, 150),
            'max_depth': trial.suggest_int('max_depth', 7, 25),
            'subsample': trial.suggest_float('subsample', 0.7, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
            'lambda_l1': trial.suggest_float('lambda_l1', 1e-5, 10.0, log=True),
            'lambda_l2': trial.suggest_float('lambda_l2', 1e-5, 10.0, log=True),
        }

        # TimeSeriesSplit 교차 검증을 적용합니다.
        tscv = TimeSeriesSplit(n_splits=5)
        rmse_scores = []
        
        # --- for 반복문 내부에 실제 실행 코드 추가 ---
        for train_idx, val_idx in tscv.split(X):
            # 1. 데이터를 훈련 및 검증 세트로 분리합니다.
            X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
            y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

            # 2. 모델을 생성하고 훈련합니다.
            model = lgb.LGBMRegressor(**param)
            model.fit(X_train_fold, y_train_fold,
                      eval_set=[(X_val_fold, y_val_fold)],
                      eval_metric='rmse',
                      callbacks=[lgb.early_stopping(50, verbose=False)])
            
            # 3. 검증 세트로 예측 후 RMSE를 계산하여 리스트에 추가합니다.
            val_preds = model.predict(X_val_fold)
            rmse = np.sqrt(mean_squared_error(y_val_fold, val_preds))
            rmse_scores.append(rmse)
        
        # 5개 구간에서 얻은 RMSE 점수의 평균을 반환합니다.
        return np.mean(rmse_scores)

    # Optuna 최적화를 실행합니다.
    if 'X_train_selected' in locals():
        logger.write(">> [7단계 시작] Optuna와 TimeSeriesSplit으로 하이퍼파라미터 최적화를 시작합니다...")
        
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective(trial, X_train_selected, y_train), n_trials=50)
        
        # 탐색을 통해 찾은 최적의 파라미터를 저장합니다.
        best_params = study.best_params
        best_params.update({
            'objective': 'regression_l1', 'metric': 'rmse', 'n_estimators': 2000,
            'verbosity': -1, 'seed': 42, 'device': 'cuda'
        })
        
        logger.write(f">> Optuna 탐색 완료. 최적 파라미터: {best_params}")
        logger.write(">> [7단계 완료] 하이퍼파라미터 최적화 성공.")

except Exception as e:
    logger.write(f">> [오류] 7단계 하이퍼파라미터 최적화 중 문제 발생: {e}", print_error=True)


In [ ]:
# --- 8. 최종 모델 훈련 및 예측 ---
# Optuna를 통해 찾은 최적의 하이퍼파라미터로 최종 모델을 훈련하고, 테스트 데이터에 대한 예측을 수행합니다.

try:
    if 'best_params' in locals():
        logger.write(">> [8단계 시작] 최종 모델 훈련 및 예측을 시작합니다...")
        
        # 최종 LightGBM 모델을 최적의 파라미터로 초기화합니다.
        final_model = lgb.LGBMRegressor(**best_params)
        
        # 선택된 특성으로 구성된 전체 훈련 데이터를 사용하여 모델을 훈련합니다.
        final_model.fit(X_train_selected, y_train)
        
        # 테스트 데이터에 대한 예측을 수행합니다.
        predictions = final_model.predict(X_test_selected)
        
        # [중요] 로그 변환되었던 예측값을 원래의 가격 스케일로 되돌립니다.
        # np.expm1은 np.exp(x) - 1과 같으며, np.log1p의 역함수 관계입니다.
        predictions = np.expm1(predictions)
        
        logger.write(">> [8단계 완료] 최종 모델 훈련 및 예측 성공.")
    else:
        logger.write(">> [알림] 7단계 하이퍼파라미터 최적화가 실행되지 않아 8단계를 건너뜁니다.")
        
except Exception as e:
    logger.write(f">> [오류] 8단계 최종 모델 훈련 및 예측 중 문제 발생: {e}", print_error=True)

In [ ]:
# --- 9. 제출 파일 생성 ---
# 예측 결과를 경진대회 제출 형식에 맞게 CSV 파일로 저장합니다.

try:
    if 'predictions' in locals():
        logger.write(">> [9단계 시작] 제출 파일 생성을 시작합니다...")
        
        # 모델 저장
        joblib.dump(final_model, MODEL_PATH)
        logger.write(f">> 모델 저장 완료: {MODEL_PATH}")
        
        logger.write(">> 'target' 컬럼만 포함된 제출 파일을 생성합니다.")
        # 제출용 샘플 파일의 'target' 컬럼에 예측값을 채워 넣습니다.
        submission_df['target'] = predictions
        submission_df['target'] = submission_df['target'].astype(int)
        
        # 'submission.csv' 파일로 저장합니다. 인덱스는 저장하지 않습니다.
        submission_df.to_csv(SUBMISSION_PATH, index=False)
        logger.write(f">> 제출 파일 생성 완료: {SUBMISSION_PATH}")
        
        logger.write(">> [9단계 완료] 제출 파일 생성 성공. 'submission.csv' 파일을 확인하세요.")
        
except Exception as e:
    logger.write(f">> [오류] 9단계 제출 파일 생성 중 문제 발생: {e}", print_error=True)

In [ ]:
# --- 10. 최종 모델 검증 및 시각화 ---
# 모델의 성능을 다각도로 분석하고, 예측의 근거를 확인합니다.

try:
    if 'final_model' in locals():
        logger.write(">> [10단계 시작] 최종 모델 검증 및 시각화를 시작합니다...")

        # --- 1. OOF 예측 생성 및 CV 점수 확보 (신뢰도 높은 분석을 위함) ---
        logger.write(">> 1. OOF(Out-of-Fold) 예측값 및 CV 점수를 생성합니다...")
        
        # TimeSeriesSplit 설정
        tscv = TimeSeriesSplit(n_splits=5)
        
        oof_preds = np.zeros(X_train_selected.shape[0]) # OOF 예측을 저장할 배열
        cv_rmse_scores = [] # 각 Fold의 RMSE 점수를 저장할 리스트

        # 교차 검증을 다시 수행하여 OOF 예측 생성
        for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train_selected)):
            X_train_fold, X_val_fold = X_train_selected.iloc[train_idx], X_train_selected.iloc[val_idx]
            y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
            
            # best_params로 모델을 다시 훈련
            model = lgb.LGBMRegressor(**best_params)
            model.fit(X_train_fold, y_train_fold,
                      eval_set=[(X_val_fold, y_val_fold)],
                      eval_metric='rmse',
                      callbacks=[lgb.early_stopping(50, verbose=False)])
            
            # 검증 세트(Validation Set)에 대한 예측값을 OOF 배열의 해당 위치에 저장
            val_preds = model.predict(X_val_fold)
            oof_preds[val_idx] = val_preds
            
            # 현재 Fold의 RMSE 점수 계산 및 저장
            rmse = np.sqrt(mean_squared_error(y_val_fold, val_preds))
            cv_rmse_scores.append(rmse)
            logger.write(f">> Fold {fold+1} RMSE: {rmse:.4f}")

        logger.write(f">> 최종 CV 평균 RMSE: {np.mean(cv_rmse_scores):.4f}")
        
        # --- 2. 피처 중요도(Feature Importance) 시각화 ---
        logger.write("\n>> 2. 피처 중요도를 시각화합니다...")
        feature_importances = pd.DataFrame(
            {'feature': X_train_selected.columns, 'importance': final_model.feature_importances_}
        ).sort_values('importance', ascending=False)

        plt.figure(figsize=(12, 10))
        sns.barplot(x='importance', y='feature', data=feature_importances.head(20))
        plt.title('상위 20개 피처 중요도 (Feature Importance)', fontsize=16)
        plt.xlabel('중요도', fontsize=12)
        plt.ylabel('피처', fontsize=12)
        plt.grid(True, axis='x', linestyle='--')
        plt.tight_layout()
        plt.show()

        # --- 3. 실제 값 vs OOF 예측 값 비교 (Scatter Plot) ---
        logger.write("\n>> 3. 실제 값과 OOF 예측 값을 비교합니다...")
        plt.figure(figsize=(10, 10))
        sns.scatterplot(x=y_train, y=oof_preds, alpha=0.3)
        plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2, label='이상적인 예측선 (y=x)')
        plt.title('실제 값 vs OOF 예측 값 비교', fontsize=16)
        plt.xlabel('실제 값 (Actual)', fontsize=12)
        plt.ylabel('OOF 예측 값 (Predicted)', fontsize=12)
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # --- 4. 잔차(Residuals) 분석 ---
        logger.write("\n>> 4. 잔차(실제-예측)를 분석합니다...")
        residuals = y_train - oof_preds
        
        # 잔차 분포 (Histogram)
        plt.figure(figsize=(10, 6))
        sns.histplot(residuals, kde=True, bins=50)
        plt.title('잔차 분포 (OOF)', fontsize=16)
        plt.xlabel('잔차 (Residuals)', fontsize=12)
        plt.ylabel('빈도 (Frequency)', fontsize=12)
        plt.axvline(x=0, color='red', linestyle='--')
        plt.grid(True)
        plt.tight_layout()
        plt.show()
        
        # 잔차 vs 예측 값 (Scatter Plot)
        plt.figure(figsize=(10, 6))
        sns.scatterplot(x=oof_preds, y=residuals, alpha=0.3)
        plt.axhline(y=0, color='red', linestyle='--')
        plt.title('잔차 vs OOF 예측 값', fontsize=16)
        plt.xlabel('OOF 예측 값 (Predicted)', fontsize=12)
        plt.ylabel('잔차 (Residuals)', fontsize=12)
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # --- 5. 교차 검증(CV) 폴드별 RMSE 점수 시각화 ---
        logger.write("\n>> 5. 교차 검증 폴드별 RMSE 점수를 시각화합니다...")
        plt.figure(figsize=(10, 6))
        sns.barplot(x=[f'Fold {i+1}' for i in range(len(cv_rmse_scores))], y=cv_rmse_scores)
        plt.title('TimeSeriesSplit 교차 검증 RMSE 점수', fontsize=16)
        plt.xlabel('폴드 (Fold)', fontsize=12)
        plt.ylabel('RMSE', fontsize=12)
        plt.axhline(y=np.mean(cv_rmse_scores), color='r', linestyle='--', label=f'평균 RMSE: {np.mean(cv_rmse_scores):.4f}')
        plt.legend()
        plt.grid(True, axis='y')
        plt.tight_layout()
        plt.show()

        # --- 6. 학습 데이터와 테스트 예측 결과 분포 비교 (KDE Plot) ---
        logger.write("\n>> 6. 학습 데이터와 테스트 예측의 분포를 비교합니다...")
        plt.figure(figsize=(10, 6))
        # 로그 변환된 값으로 비교해야 스케일이 맞음
        sns.kdeplot(y_train, label='학습 데이터 실제 값 (Log)', color='blue', fill=True)
        sns.kdeplot(np.log1p(predictions), label='테스트 데이터 예측 값 (Log)', color='orange', fill=True)
        plt.title('학습 데이터와 테스트 예측의 분포 비교 (Log Scale)', fontsize=16)
        plt.xlabel('아파트 가격 (Log Transformed)', fontsize=12)
        plt.ylabel('밀도 (Density)', fontsize=12)
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # --- 7. SHAP (SHapley Additive exPlanations) 분석 ---
        logger.write("\n>> 7. SHAP 분석을 시작합니다 (계산에 시간이 다소 소요될 수 있습니다)...")
        try:
            explainer = shap.TreeExplainer(final_model)
            # 데이터가 너무 크면 샘플링하여 사용 (1000개 샘플)
            shap_sample = X_train_selected.sample(1000, random_state=42)
            shap_values = explainer.shap_values(shap_sample)

            # SHAP 요약 플롯 (Summary Plot)
            logger.write(">> SHAP 요약 플롯: 각 피처가 예측에 미치는 영향을 종합적으로 보여줍니다.")
            shap.summary_plot(shap_values, shap_sample, plot_type="dot")

            # SHAP 의존성 플롯 (Dependence Plot)
            logger.write("\n>> SHAP 의존성 플롯: '전용면적'이 예측 가격에 미치는 영향을 보여줍니다.")
            shap.dependence_plot('전용면적', shap_values, shap_sample, interaction_index="건물나이")

        except Exception as e:
            logger.write(f">> [오류] SHAP 분석 중 문제가 발생했습니다: {e}", print_error=True)

        logger.write("\n>> [10단계 완료] 모델 분석 및 시각화 성공.")

    else:
        logger.write(">> [알림] 최종 모델이 생성되지 않아 10단계 시각화를 건너뜁니다.")

except Exception as e:
    logger.write(f">> [오류] 10단계 모델 분석 및 시각화 중 문제 발생: {e}", print_error=True)
    
# finally:
#         logger.stop_redirect()
#         logger.write(">> 모델링 종료")
#         logger.write("="*50 + "\n")
#         logger.close()